<a href="https://colab.research.google.com/github/Pmilivojevic/PyTorch/blob/main/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.nn.modules import padding

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels, **kwargs):
    super(ConvBlock, self).__init__()

    self.relu = nn.ReLU()
    self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
    self.batchnorm = nn.BatchNorm2d(out_channels)

  def forward(self, x):
    return self.relu(self.batchnorm(self.conv(x)))

In [ ]:
class InceptionBlock(nn.Module):
  def __init__(
      self,
      in_channels,
      out_1x1,
      red_3x3,
      out_3x3,
      red_5x5,
      out_5x5,
      out_1x1_pool
  ):
    super(InceptionBlock, self).__init__()

    self.branch1 = ConvBlock(in_channels, out_1x1, kernel_size=1)

    self.branch2 = nn.Sequential(
        ConvBlock(in_channels, red_3x3, kernel_size=1),
        ConvBlock(red_3x3, out_3x3, kernel_size=3, padding=1)
    )

    self.branch3 = nn.Sequential(
        ConvBlock(in_channels, red_5x5, kernel_size=1),
        ConvBlock(red_5x5, out_5x5, kernel_size=5, padding=2)
    )

    self.branch4 = nn.Sequential(
        nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
        ConvBlock(in_channels, out_1x1_pool, kernel_size=1)
    )

  def forward(self, x):
    return torch.cat([
        self.branch1(x),
        self.branch2(x),
        self.branch3(x),
        self.branch4(x)
      ],
      1
    )

In [ ]:
class GoogLeNet(nn.Module):
  def __init__(self, in_channels=3, num_classes=1000):
    super(GoogLeNet, self).__init__()

    self.conv1 = ConvBlock(
        in_channels=in_channels,
        out_channels=64,
        kernel_size=7,
        stride=2,
        padding=3
    )

    self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    self.conv2 = ConvBlock(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1)

    self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
    self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)

    self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
    self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
    self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
    self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
    self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)

    self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
    self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)

    self.avg_pool = nn.AvgPool2d(kernel_size=7, stride=1)

    self.drop = nn.Dropout(0.4)

    self.fc = nn.Linear(1024, num_classes, )

  def forward(self, x):
    x = self.conv1(x)
    x = self.max_pool(x)
    x = self.conv2(x)
    x = self.max_pool(x)

    x = self.inception3a(x)
    x = self.inception3b(x)
    x = self.max_pool(x)

    x = self.inception4a(x)
    x = self.inception4b(x)
    x = self.inception4c(x)
    x = self.inception4d(x)
    x = self.inception4e(x)
    x = self.max_pool(x)

    x = self.inception5a(x)
    x = self.inception5b(x)
    x = self.avg_pool(x)
    x = x.reshape(x.shape[0], -1)

    x = self.drop(x)
    x = self.fc(x)

    return x

In [ ]:
model = GoogLeNet()
x = torch.randn(3, 3, 224, 224)
print(model(x).shape)

torch.Size([3, 1000])
